<a href="https://colab.research.google.com/github/ramenwang/deep_learning_py/blob/master/super_resolution/RSSuperRes_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from keras.layers import Concatenate, Conv2D, LeakyReLU, Dense, Lambda
from keras.layers import BatchNormalization, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:

def conv2d(x, num_filter, filter_size, stride=(1,1), padding='same', name=None,
           act=True, batchnorm=True, dropout_rate=None):
    ''' conv2d with optional batchnorm, leakyReLU activation, or dropout
    '''
    x = Conv2D(filters=num_filter, kernel_size=filter_size, strides=stride,
               padding=padding, name=name+'_conv2d')(x)
    # batch normalization
    if (batchnorm):
        x = BatchNormalization(name=name+'_conv2d_bn')(x)
    # activation
    if (act):
        x = LeakyReLU(name=name+'_conv2d_leakyReLU')(x)
    # random dropout
    if (dropout_rate != None):
        x = Dropout(rate=dropout_rate)(x)
    
    return x


def inception_module(x, nin_filter1, nin_filter2, dropout_rate):
    ''' inception layer with bottleneck layer to reduce the depth
    '''
    # path 1 - bottleneck layer
    p1 = conv2d(x, num_filter=nin_filter1, filter_size=(1,1), name='Inc_path_1_bn',
                act=True, batchnorm=True, dropout_rate=dropout_rate)
    # path 2 - bottleneck layer + 3*3 filter
    p2 = conv2d(x, num_filter=nin_filter1, filter_size=(1,1), name='Inc_path_2_bn',
                act=True, batchnorm=True, dropout_rate=dropout_rate)
    p2 = conv2d(p2, num_filter=nin_filter1, filter_size=(3,3), name='Inc_path_2_conv',
                act=True, batchnorm=True, dropout_rate=dropout_rate)
    # path 3 - bottleneck layer + 3*3 filter
    p3 = conv2d(x, num_filter=nin_filter1, filter_size=(1,1), name='Inc_path_3_bn',
                act=True, batchnorm=True, dropout_rate=dropout_rate)
    p3 = conv2d(p3, num_filter=nin_filter1, filter_size=(5,5), name='Inc_path_3_conv',
                act=True, batchnorm=True, dropout_rate=dropout_rate)
    
    output = Concatenate([p_1, p_2, p_3], axis=3, name='Inc_concat')
    return output
    

def pixel_shuffler(x, scale, output_filters, filter_size):
    ''' pixel shuffler layer to reconstruct the 2d space from feature map channels
    '''
    ps = conv2d(x, num_filter=output_filters*scale*scale, filter_size=filter_size,
                name='ps', batchnorm=False, act=False, dropout_rate=None)
    # using depth_to_space to resampling the image pixel from depth
    Subpixel_layer = Lambda(lambda x: tf.nn.depth_to_space(x,scale))
    ps = Subpixel_layer(inputs=ps)
    ps = LeakyReLU(name='ps_conv2d_leakyReLU')(ps)
    return ps
    

def feature_extractor(x, n_layers, filter_size, n_filter, min_filters, 
                      filter_decay_gamma, dropout_rate):
    ''' building feature extractor with skip connection and filter gamma decay
    '''
    concat_list = []
    n_filter_list = []
    for i in range(n_layers):
        # perform gamma decay
        if min_filters !=0 and i > 0:
            x1 = i / float(n_layers - 1)
            y1 = pow(x1, 1. / filter_decay_gamma)
            n_filters = int((n_filters - min_filters) * (1 - y1) + min_filters)
            n_filter_list.append(n_filters)
        # perform convolution
        x = conv2(x, num_filter=n_filters, filter_size=filter_size, name='FE_'+str(i),
                  act=True, batchnorm=True, dropout_rate=dropout_rate)
        concat_list.append(x)

    output = Concatenate(concat_list, axis=3, name='FE_concat')
    return output, n_filter_list


def RSSuperRes(x, bicubic_x, n_layers, min_filters, filter_decay_gamma, 
               dropout_rate, n_reconstructors, num_recon_filter, output_channels):
    ''' super resolution model
    '''
    x = feature_extractor(x=x, n_layers, filter_size=3, n_filter, min_filters, 
                          filter_decay_gamma, dropout_rate)
    x = inception_module(x=x, nin_filter1=nin_filter1, nin_filter2=nin_filter2,
                         dropout_rate=dropout_rate)
    x = pixel_shuffler(x=x, scale=scale, output_filters=output_filters,
                       filter_size=filter_size)
    for i in range(max(1, n_reconstructors)):
        x = conv2d(x=x, num_filter=num_recon_filter, filter_size=filter_size, batchnorm=False,
                   dropout_rate=dropout_rate)
    x = conv2d(x=x, num_filter=output_channels, filter_size=filter_size, batchnorm=False,
               dropout_rate=dropout_rate, act=False, name='output')

    x = x + bicubic_x

    return x

2.1.0
